In [ ]:
import pandas as pd
df = pd.read_csv('../input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv',
                index_col=0)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['Label'].unique()

In [ ]:
df[df['Label'] == 'Pnemonia']

In [ ]:
df[df['Label'] == 'Normal']

In [ ]:
df[df['X_ray_image_name'] == 'IM-0675-0001.jpeg']

In [ ]:
df.shape

In [ ]:
import os    #Python Library to access files within a directory
os.listdir('../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train')

In [ ]:
len(os.listdir('../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'))

In [ ]:
from keras.preprocessing.image import load_img,array_to_img

In [ ]:
load_img('../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/person1590_bacteria_4175.jpeg')

In [ ]:
loc = '../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'
print(loc)

In [ ]:
os.listdir(loc)

In [ ]:
k1 = 'person1180_virus_2010.jpeg'
k2 = 'person588_bacteria_2422.jpeg'

k3 = 'NORMAL2-IM-1169-0001.jpeg'
k4 = 'IM-0421-0001.jpeg'

print(k1.split('_'))
print(k2.split('_'))
print(k3.split('_'))
print(k4.split('_'))

In [ ]:
print(len(k1.split('_')) > 1)
print(len(k2.split('_')) > 1)
print(len(k3.split('_')) > 1)
print(len(k4.split('_')) > 1)

****Creating Label matrix using name of images given****

In [ ]:
labels = []
for names in os.listdir(loc):
    if len(names.split('_')) > 1:
        labels.append(1)
    else:
        labels.append(0)

In [ ]:
import numpy as np
Y = np.array(labels)

**Create the feature matrix using the image given**

In [ ]:
#openCV - for image preprocessing and creating feature matrix
import cv2
im = cv2.imread('../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/person1590_bacteria_4175.jpeg',0)
imr = cv2.resize(im,(70,70))

In [ ]:
imr

In [ ]:
imr.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(imr)
plt.show()

In [ ]:
array_to_img(imr.reshape(70,70,1))

In [ ]:
loc = '../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'
features = []
from tqdm import tqdm

for image in tqdm(os.listdir(loc)):
    f = cv2.imread(os.path.join(loc,image),0)
    fr = cv2.resize(f,(70,70))
    features.append(fr)

In [ ]:
X = np.array(features)
X.shape

In [ ]:
ft = X.reshape(5309,4900)
lb = Y.reshape(5309,1)

**Create Data Frame of extracted Images**

In [ ]:
import pandas as pd
ft = pd.DataFrame(ft)
lb = pd.DataFrame(lb)
lb.columns = ['Label']

In [ ]:
coronahack = pd.concat((lb,ft),axis=1)
coronahack.to_csv('coranahack.csv')

**Create training and validation sets**

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y)

In [ ]:
xtrain = xtrain.reshape(3981,4900)
xtest = xtest.reshape(1328,4900)

In [ ]:
print(xtest.shape)

In [ ]:
from keras import models
from keras import layers
from keras import regularizers
from keras.utils import np_utils

In [ ]:
#normalize your features
xtrain = xtrain/xtrain.max()
xtest = xtest/xtest.max()

In [ ]:
xtrain.shape[1]

In [ ]:
#One hot encoding of labels
ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)

**Create your neural network model**

In [ ]:
wd = 1e-4
model = models.Sequential()

model.add(layers.Dense(512 , activation='relu' , kernel_regularizer = regularizers.l2(wd) , input_dim=xtrain.shape[1]))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(256 , activation='relu' , kernel_regularizer = regularizers.l2(wd)))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(128 , activation='relu' , kernel_regularizer = regularizers.l2(wd)))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(2 , activation='sigmoid'))

In [ ]:
model.summary()

**Compile Your model**

In [ ]:
from keras import optimizers,metrics

model.compile(optimizer='sgd',
             loss = 'categorical_crossentropy',
             metrics = ['Accuracy'])

**Train Your model**

In [ ]:
model.fit(xtrain,ytrain,
          epochs=15,
          validation_data=(xtest,ytest))